In [ ]:
import cv2
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, BatchNormalization
from sklearn.preprocessing import OneHotEncoder
import pandas as pad
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import keras

In [ ]:
from keras.datasets import mnist

# Download the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
import numpy as np
import random

In [ ]:
def RBF(x, c, s):
    return np.exp(-np.sum((x-c)**2, axis=1)/(2*s**2))


In [ ]:
# Randomly sample the training set without replacement
num_samples = int(len(x_train) / 10)
x_train_reduced = x_train[random.sample(range(len(x_train)), num_samples)]
y_train_reduced = y_train[random.sample(range(len(y_train)), num_samples)]

In [ ]:
print("Train data shape: ", x_train.shape, y_train.shape)
print("Test data shape: ", x_test.shape, y_test.shape)

Train data shape:  (60000, 28, 28) (60000,)
Test data shape:  (10000, 28, 28) (10000,)


In [ ]:
print("Train Data Reduced shape: ", x_train_reduced.shape, y_train_reduced.shape)

Train Data Reduced shape:  (6000, 28, 28) (6000,)


In [ ]:
def transform(image):
    image = np.pad(image, (2, 2))
    c = np.mean(image)
    s = np.std(image)
    return RBF(image, c, s).flatten()

In [ ]:
x_train_tf = []
for image in x_train:
    x_train_tf.append(transform(image))
x_train_tf = np.array(x_train_tf)
print("Shape of x_train after transforming: ", x_train_tf.shape)
x_test_tf = []
for image in x_test:
    x_test_tf.append(transform(image))
x_test_tf = np.array(x_test_tf)
print("Shape of x_test after transforming: ", x_test_tf.shape)


Shape of x_train after transforming:  (60000, 32)
Shape of x_test after transforming:  (10000, 32)


In [ ]:
def split_train_test_val(x_train_tf, train_size=0.8, test_size=0.1, val_size=0.1):
  """Splits a NumPy array into training, testing, and validation datasets.

  Args:
    x_train_tf: A NumPy array to split.
    train_size: The proportion of the data to use for training.
    test_size: The proportion of the data to use for testing.
    val_size: The proportion of the data to use for validation.

  Returns:
    A tuple of three NumPy arrays: training data, testing data, and validation
    data.
  """

  # Calculate the number of samples in each dataset.
  n_train = int(train_size * len(x_train_tf))
  n_test = int(test_size * len(x_train_tf))
  n_val = int(val_size * len(x_train_tf))

  # Shuffle the data.
  np.random.shuffle(x_train_tf)

  # Split the data into training, testing, and validation datasets.
  train = x_train_tf[:n_train]
  test = x_train_tf[n_train:n_train + n_test]
  val = x_train_tf[n_train + n_test:]

  return train, test, val

In [ ]:

encoder = OneHotEncoder()

y_train = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test = encoder.transform(y_test.reshape(-1, 1)).toarray()

print("Shape of y_train after OHE: ", y_train.shape)
print("Shape of y_test after OHE: ", y_test.shape)

Shape of y_train after OHE:  (60000, 10)
Shape of y_test after OHE:  (10000, 10)


In [ ]:
print(tf.__version__)

2.14.0


### 4.1 Activation function : Sigmoid | Hidden Neurons : [16]


In [ ]:
tf.keras.backend.clear_session()

input_layer = Input(shape=(32, ))
dense_layer = Dense(16, activation='sigmoid')(input_layer)
output_layer = Dense(10, activation='softmax',
                     kernel_initializer='glorot_normal')(dense_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense (Dense)               (None, 16)                528       
                                                                 
 dense_1 (Dense)             (None, 10)                170       
                                                                 
Total params: 698 (2.73 KB)
Trainable params: 698 (2.73 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
!mkdir model1_1
model_path = "model1_1/"

mkdir: cannot create directory ‘model1_1’: File exists


In [ ]:
adam = Adam(learning_rate=0.001)

In [ ]:
# Setting the filepath for the ModelCheckpoint callback to save the files
filepath = model_path+"{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(
    filepath=filepath, monitor='val_loss',  verbose=1, save_best_only=True, mode='auto')

# We will use the EarlyStopping callback to prevent the model from training after the optimal condition has been met
earlystop = EarlyStopping(
    monitor='val_loss', patience=7, verbose=1, mode='min')

# Passing validation data into the custom Metrics callback that we have created
validation_data = (x_test_tf, y_test)

model.compile(optimizer=adam, loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# !rm -rf ./logs1_1/fit

In [ ]:

log_dir = "logs1_1/fit/" + datetime.now().strftime("%Y%m%d - %H%M%S")

tf.keras.utils.plot_model(model, to_file=model_path +
                          'model.png', show_shapes=True)

tbCallBack = TensorBoard(log_dir=log_dir, write_graph=True,
                         write_grads=True, write_images=True)

allCs = [tbCallBack, earlystop, checkpoint]

model.fit(x_train_tf, y_train, epochs=50,
          validation_data=validation_data, batch_size=256, callbacks=allCs)

Epoch 1/50
234/235 [============================>.] - ETA: 0s - loss: 2.2956 - accuracy: 0.1716
Epoch 1: val_loss improved from inf to 2.24427, saving model to model1_1/01.hdf5
235/235 [==============================] - 4s 9ms/step - loss: 2.2955 - accuracy: 0.1718 - val_loss: 2.2443 - val_accuracy: 0.2247
Epoch 2/50
  9/235 [>.............................] - ETA: 1s - loss: 2.2447 - accuracy: 0.2088

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


232/235 [============================>.] - ETA: 0s - loss: 2.1966 - accuracy: 0.2517
Epoch 2: val_loss improved from 2.24427 to 2.14137, saving model to model1_1/02.hdf5
235/235 [==============================] - 2s 9ms/step - loss: 2.1961 - accuracy: 0.2519 - val_loss: 2.1414 - val_accuracy: 0.2883
Epoch 3/50
213/235 [==========================>...] - ETA: 0s - loss: 2.0846 - accuracy: 0.3038
Epoch 3: val_loss improved from 2.14137 to 2.02112, saving model to model1_1/03.hdf5
235/235 [==============================] - 1s 4ms/step - loss: 2.0798 - accuracy: 0.3067 - val_loss: 2.0211 - val_accuracy: 0.3339
Epoch 4/50
231/235 [============================>.] - ETA: 0s - loss: 1.9600 - accuracy: 0.3560
Epoch 4: val_loss improved from 2.02112 to 1.90659, saving model to model1_1/04.hdf5
235/235 [==============================] - 1s 3ms/step - loss: 1.9596 - accuracy: 0.3559 - val_loss: 1.9066 - val_accuracy: 0.3504
Epoch 5/50
213/235 [==========================>...] - ETA: 0s - loss: 1.853

In [ ]:
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir ./logs1_1/

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2023-10-31 05:32:46.716475: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-31 05:32:46.716549: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-31 05:32:46.716578: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-31 05:32:48.265577: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorf

 ### 4.2 Activation function: Sigmoid; Hidden neurons: [32, 16]

In [ ]:
tf.keras.backend.clear_session()

input_layer = Input(shape=(1024, ))
dense_layer_1 = Dense(32, activation='sigmoid')(input_layer)
dense_layer_2 = Dense(16, activation='sigmoid')(dense_layer_1)
output_layer = Dense(10, activation='softmax',
                     kernel_initializer='glorot_normal')(dense_layer_2)

model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1024)]            0         
                                                                 
 dense (Dense)               (None, 32)                32800     
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 10)                170       
                                                                 
Total params: 33498 (130.85 KB)
Trainable params: 33498 (130.85 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
!mkdir model1_2
model_path = "model1_2/"

mkdir: cannot create directory ‘model1_2’: File exists


In [ ]:
model.compile(optimizer=adam, loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
!rm -rf ./logs1_2/fit
tf.keras.backend.clear_session()


In [ ]:
log_dir = "logs1_2/fit/" + datetime.now().strftime("%Y%m%d - %H%M%S")

In [ ]:
tf.keras.utils.plot_model(model, to_file=model_path +
                          'model.png', show_shapes=True)

tbCallBack = TensorBoard(log_dir=log_dir, write_graph=True,
                         write_grads=True, write_images=True)

allCs = [tbCallBack, earlystop]

model.fit(x_train_tf, y_train, epochs=50,
          validation_data=validation_data, batch_size=256, callbacks=allCs)

Epoch 1/50


ValueError: ignored

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs1_2